In [1]:
import os
import re
import tensorflow as tf

In [2]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
!git lfs install

Git LFS initialized.


In [347]:
!git config --global user.email "denizhan"
!git config --global user.name "denizzhansahin"

In [348]:
!git config --global credential.helper store

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [328]:
%cd /content/drive/MyDrive/Veri-Kazima/Model_Eğitim

/content/drive/MyDrive/Veri-Kazima/Model_Eğitim


In [329]:
#veri setini yükle
file_name = "Tum-Veri-Seti-Nisan2024-10.csv"

In [330]:
#veri seti hakkında bilgi al
import pandas as pd

def kelime_sayisi(dosya_adi):
  with open(dosya_adi, "r") as dosya:
    metin = dosya.read()
  kelimeler = metin.split()
  return len(kelimeler)

dosya_kaydet = file_name
data = pd.read_csv(dosya_kaydet)
data = pd.DataFrame(data)

In [331]:
data.Icerik.count()

94009

In [332]:
len(data)

113814

In [333]:
data['Icerik'].fillna(' ', inplace=True)

In [334]:
data.Icerik.count()

113814

In [335]:
file_name = "/content/drive/MyDrive/Veri-Kazima/Model_Eğitim/Yuklenen/"+file_name

In [336]:
data.to_csv(file_name)

In [337]:
from datasets import load_dataset
reviews_dataset = load_dataset("csv", data_files= file_name)

Generating train split: 0 examples [00:00, ? examples/s]

In [338]:
#veri setini 4000 değerine kadar al
reviews_sample = reviews_dataset["train"].shuffle(seed=100).select(range(data.Icerik.count()))

In [339]:
reviews_sample

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'Baslik', 'Ozet', 'Kategori', 'Link', 'Icerik'],
    num_rows: 113814
})

In [340]:
reviews_sample = reviews_sample.remove_columns('Unnamed: 0.1')

In [341]:
#her bir satır değerini ilgili sütun için split et
def compute_review_length(example):
    return {"review_length": len(example["Icerik"].split())}

In [342]:
#veriler için düzenleme yap, gereksiz olanları değiştir veya kaldır
def remove_repeated(example):
    example["Icerik"] = example["Icerik"].replace('...', '')
    example["Icerik"] = example["Icerik"].replace(',"', '. ')
    example["Icerik"] = example["Icerik"].replace('!.', '.')
    example["Icerik"] = example["Icerik"].replace('!,', '. ')
    example["Icerik"] = example["Icerik"].replace('"', '')
    example["Icerik"] = re.sub('([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ]),([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ])', '\\1. \\2', example["Icerik"])

    return {"Icerik": example["Icerik"].replace('Devamını oku', '')}

In [343]:
reviews_sample = reviews_sample.map(remove_repeated)
reviews_sample[:3]

Map:   0%|          | 0/113814 [00:00<?, ? examples/s]

{'Unnamed: 0': [82773, 97819, 62555],
 'Baslik': ['Facebook, Myanmar ordusuna ait resmi hesabı sildi',
  'Sirkeci-Kazlıçeşme Raylı Sistem Hattı yarın açılacak',
  'Hakkı Alkan’a nasıl Çince öğrettik? Yapay zeka işleri değiştiriyor!'],
 'Ozet': ['Facebook, Myanmar ordusuna ait sayfayı, topluluk kurallarını ihlal ettiği gerekçesiyle kapattı. Sosyal medya devi, ilgili sayfanın ordu propagandası yaptığını belirterek açıklama yaptı.',
  "Sirkeci-Kazlıçeşme Raylı Sistem ve Yaya Odaklı Yeni Nesil Ulaşım Projesi, Cumhurbaşkanı Erdoğan'ın katılımıyla yarın hizmete alınacak.",
  'Sağlık, finans, üretim, ulaşım ve savunma gibi sektörlerde kullanımı giderek yaygınlaşan yapay zekayı web alanında nasıl kullanabiliriz?'],
 'Kategori': ['TEKNOLOJI HABERLERI', 'Gündem', 'SDNTV'],
 'Link': ['https://shiftdelete.net/facebook-myanmar-ordusuna-ait-resmi-hesabi-sildi',
  'https://www.trthaber.com/haber/gundem/sirkeci-kazlicesme-rayli-sistem-hatti-yarin-acilacak-839993.html',
  'https://shiftdelete.net/ticim

In [344]:
#veri setini bölümle
reviews_sample = reviews_sample.train_test_split(train_size=0.7, seed=100)
# test sütununu validation sütunu olarak ayarla
reviews_sample["validation"] = reviews_sample.pop("test")

reviews_sample

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Baslik', 'Ozet', 'Kategori', 'Link', 'Icerik'],
        num_rows: 79669
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Baslik', 'Ozet', 'Kategori', 'Link', 'Icerik'],
        num_rows: 34145
    })
})

In [346]:
#veri setini yükle
#reviews_sample.push_to_hub("kullanıcı adı/alan adı")
reviews_sample.push_to_hub("denizzhansahin/99K-TR-News")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/80 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/600 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/denizzhansahin/99K-TR-News/commit/d8341ec24d45f52de256eeb6009a3781053e058e', commit_message='Upload dataset', commit_description='', oid='d8341ec24d45f52de256eeb6009a3781053e058e', pr_url=None, pr_revision=None, pr_num=None)